In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from utils.data import load_tc_data
from utils.plot import make_cmap
import cftime as cf



In [2]:
load_from_scratch=True
plt.rcParams.update({
    # Use LaTeX default serif font-family.
    # https://matplotlib.org/stable/tutorials/text/usetex.html#sphx-glr-tutorials-text-usetex-py
     "font.family": "sans-serif",
     # Use specific cursive fonts.
     "font.cursive": ["Comic Neue", "Comic Sans MS"],
 })
font = {'fontname':'Comic Neue'}

In [3]:
# tracks = pd.read_csv('/user/home/al18709/work/ibtracks/ibtracs.ALL.list.v04r00.csv')

In [4]:
def find_dates(sids,model_cal):

	dates = sids.apply(lambda row : cf.datetime(calendar=model_cal,
									year=row.year,
									month=row.month,
									day=row.day,
									hour=row.hour
									), axis=1)
	return dates



In [58]:
def lookup(row,cal):
	# date = cf.datetime(calendar=cal,
	# 				year=row.year,
	# 				month=row.month,
	# 				day=row.day,
	# 				hour=row.hour
	# 				)

	date = cf.datetime(calendar=cal,
					year=row.year,
					month=row.month,
					day=row.day,
					)

	# date = pd.to_datetime('year' : row.year, 'month' : row.month, 'day' = row.day)
	return date

def lookup_ibtracs(row,tracks):
	tracks = pd.read_csv('/user/work/al18709/ibtracks/ibtracs.ALL.list.v04r00.csv',
						usecols=['SID','LAT','LON','BASIN','NAME','SEASON', 'NATURE','ISO_TIME','USA_SSHS'],
						parse_dates = ['ISO_TIME'],keep_default_na=False)
	boolean = (tracks.SID == row.sid) & (tracks.LAT == row.centre_lat) & (tracks.LON == row.centre_lon)
	time =  pd.to_datetime(tracks.loc[boolean].ISO_TIME)
	if list(time) == []:
		print('no',flush=True)
		date = cf.datetime(calendar='gregorian',
						year=1978,
						month=1,
						day=1,
						hour=0
						)
		return date
	else:
		time =  time.iloc[0]
		date = cf.datetime(calendar='gregorian',
						year=time.year,
						month=time.month,
						day=time.day,
						hour=time.hour
						)
	return date

def find_dates_mswep(meta):
	dates = meta.apply(lookup, axis=1)
	return dates

def find_dates(meta,calendar):
	dates = meta.apply(lookup, cal = calendar, axis=1)
	return dates

def find_dates_ibtracs(meta):
	tracks = pd.read_csv('/user/work/al18709/ibtracks/ibtracs.ALL.list.v04r00.csv',
						usecols=['SID','LAT','LON','BASIN','NAME','SEASON', 'NATURE','ISO_TIME','USA_SSHS'],
						parse_dates = ['ISO_TIME'],keep_default_na=False)
	dates = meta.apply(lookup_ibtracs, tracks= tracks, axis=1)
	# dates = meta.apply(lookup_ibtracs, axis=1)
	return dates

In [ ]:

for i in range (len(meta)):

	row = meta.iloc[i]

	boolean = (tracks.SID == row.sid) & (tracks.LAT == row.centre_lat) & (tracks.LON == row.centre_lon)
	time =  pd.to_datetime(tracks.loc[boolean].ISO_TIME)

	print(time)

	if list(time) == []:
		print('hello')
		continue
	time =  time.iloc[0]
	print(time)
	date = cf.datetime(calendar='gregorian',
							year=time.year,
							month=time.month,
							day=time.day,
							hour=time.hour
	)
	print(date)

In [ ]:
# load current 1D dataset
real,inputs,pred,meta = load_tc_data(set='validation',results='ke_tracks')
tracks = pd.read_csv('/user/work/al18709/ibtracks/ibtracs.ALL.list.v04r00.csv',
						usecols=['SID','LAT','LON','BASIN','NAME','SEASON', 'NATURE','ISO_TIME','USA_SSHS'],
						parse_dates = ['ISO_TIME'],keep_default_na=False)
# print(lookup_ibtracs(meta.iloc[0],tracks))
dates = find_dates_ibtracs(meta)
print(dates)
meta['date'] = dates

# load ibtracks
tracks = pd.read_csv('/user/work/al18709/ibtracks/ibtracs.ALL.list.v04r00.csv',
						usecols=['SID','LAT','LON','BASIN','NAME','SEASON', 'NATURE','ISO_TIME','USA_SSHS'],
						parse_dates = ['ISO_TIME'],keep_default_na=False)

og_tracks_grouped = tracks.groupby('sid')

In [ ]:
# load original 2D WGAN
real_2,inputs_2,pred_2,meta_2,imput_og,pred_og,meta_og = load_tc_data(set='validation',results='kh_tracks')
meta_2['date'] = find_dates_ibtracs(meta_2,'gregorian')
# meta_og['date'] = find_dates_ibtracs(meta_og,'gregorian')
real_og_x,_,_,_,_,_,pred_og_x,meta_og_x = load_tc_data(set='extreme_test',results='test')
meta_og_x['date'] = find_dates_ibtracs(meta_og_x,'gregorian')
meta_og = pd.read_csv('/user/work/al18709/tc_data_mswep_40/valid_meta.csv')
meta_og['date'] = find_dates_ibtracs(meta_og,'gregorian')
# meta_valid = pd.read_csv('/user/work/al18709/tc_data_flipped/valid_meta.csv')

In [ ]:
# load imerg
imerg_rain = np.load('/user/home/al18709/work/CMIP6/IMERG-V07/storm_rain/storm_rain_NH.npy')
imerg_lats = np.load('/user/home/al18709/work/CMIP6/IMERG-V07/storm_rain/storm_lats_NH.npy')
imerg_lons = np.load('/user/home/al18709/work/CMIP6/IMERG-V07/storm_rain/storm_lons_NH.npy')
imerg_sids = pd.read_csv('/user/home/al18709/work/CMIP6/IMERG-V07/storm_rain/storm_sid_NH.csv')
print(imerg_sids.sid.shape)
meta_imerg = pd.DataFrame({'sid':imerg_sids.sid ,'centre_lat':imerg_lats[:,4], 'centre_lon':imerg_lons[:,4],'date':find_dates_ibtracs(imerg_sids)})

In [59]:

# load HighResMIP CMCC-CM2-VHR4 historical NH
CMCC_hist_NH_rain = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_rain_NH.npy')
CMCC_hist_NH_lats = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_lats_NH.npy')
CMCC_hist_NH_lons = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_lons_NH.npy')
CMCC_hist_NH_sids = pd.read_csv('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_sid_NH.csv')
print(CMCC_hist_NH_sids.sid.shape)
meta_CMCC_hist_NH = pd.DataFrame({'sid':CMCC_hist_NH_sids.sid ,'centre_lat':CMCC_hist_NH_lats[:,4], 'centre_lon':CMCC_hist_NH_lons[:,4], 'date':find_dates(CMCC_hist_NH_sids,'365_day')})

# load HighResMIP CMCC-CM2-VHR4 historical SH
CMCC_hist_SH_rain = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_rain_SH.npy')
CMCC_hist_SH_lats = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_lats_SH.npy')
CMCC_hist_SH_lons = np.load('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_lons_SH.npy')
CMCC_hist_SH_sids = pd.read_csv('/user/home/al18709/work/CMIP6/HighResMIP/CMCC-CM2-VHR4/storm_rain/historical/storm_sid_SH.csv')
print(CMCC_hist_SH_sids.sid.shape)
meta_CMCC_hist_SH = pd.DataFrame({'sid':CMCC_hist_SH_sids.sid ,'centre_lat':CMCC_hist_SH_lats[:,4], 'centre_lon':CMCC_hist_SH_lons[:,4], 'date':find_dates(CMCC_hist_SH_sids,'365_day')})

# load HighResMIP EC-EARTH-3P-HR  historical NH
ecearthhr_hist_NH_rain = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_rain_NH.npy')
ecearthhr_hist_NH_lats = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_lats_NH.npy')
ecearthhr_hist_NH_lons = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_lons_NH.npy')
ecearthhr_hist_NH_sids = pd.read_csv('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_sid_NH.csv')
print(ecearthhr_hist_NH_sids.sid.shape)
meta_ecearthhr_hist_NH = pd.DataFrame({'sid':ecearthhr_hist_NH_sids.sid ,'centre_lat':ecearthhr_hist_NH_lats[:,4], 'centre_lon':ecearthhr_hist_NH_lons[:,4], 'date':find_dates(ecearthhr_hist_NH_sids,'proleptic_gregorian')})

# load HighResMIP EC-EARTH-3P-HR historical SH
ecearthhr_hist_SH_rain = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_rain_SH.npy')
ecearthhr_hist_SH_lats = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_lats_SH.npy')
ecearthhr_hist_SH_lons = np.load('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_lons_SH.npy')
ecearthhr_hist_SH_sids = pd.read_csv('/user/home/al18709/work/CMIP6/HighResMIP/EC-Earth3P-HR/storm_rain/historical/storm_sid_SH.csv')
print(ecearthhr_hist_SH_sids.sid.shape)
meta_ecearthhr_hist_SH = pd.DataFrame({'sid':ecearthhr_hist_SH_sids.sid ,'centre_lat':ecearthhr_hist_SH_lats[:,4], 'centre_lon':ecearthhr_hist_SH_lons[:,4], 'date':find_dates(ecearthhr_hist_SH_sids,'proleptic_gregorian')})


(5236,)
(2366,)
(131,)
(62,)


In [60]:

# load CMIP6 historical dataset 
miroc6_hist_rain = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_rain_NH.npy')
miroc6_hist_lats = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_lats_NH.npy')
miroc6_hist_lons = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_lons_NH.npy')
miroc6_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_sid_NH.csv')
meta_miroc6_hist = pd.DataFrame({'sid':miroc6_hist_sids.sid ,'centre_lat':miroc6_hist_lats[:,4], 'centre_lon':miroc6_hist_lons[:,4], 'date':find_dates(miroc6_hist_sids,'gregorian')})

miroc6_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_rain_SH.npy')
miroc6_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_lats_SH.npy')
miroc6_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_lons_SH.npy')
miroc6_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/MIROC6/storm_rain/historical/storm_sid_SH.csv')
meta_miroc6_hist_SH = pd.DataFrame({'sid':miroc6_hist_sids_SH.sid ,'centre_lat':miroc6_hist_lats_SH[:,4], 'centre_lon':miroc6_hist_lons_SH[:,4], 'date':find_dates(miroc6_hist_sids_SH,'gregorian')})

# CMIP6 MPI-ESM1-2-LR
mpi_hist_rain = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_rain_NH.npy')
mpi_hist_lats = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_lats_NH.npy')
mpi_hist_lons = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_lons_NH.npy')
mpi_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_sid_NH.csv')
meta_mpi_hist = pd.DataFrame({'sid':mpi_hist_sids.sid ,'centre_lat':mpi_hist_lats[:,4], 'centre_lon':mpi_hist_lons[:,4], 'date':find_dates(mpi_hist_sids,'proleptic_gregorian')})

mpi_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_rain_SH.npy')
mpi_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_lats_SH.npy')
mpi_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_lons_SH.npy')
mpi_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/MPI-ESM1-2-LR/storm_rain/historical/storm_sid_SH.csv')
meta_mpi_hist_SH = pd.DataFrame({'sid':mpi_hist_sids_SH.sid ,'centre_lat':mpi_hist_lats_SH[:,4], 'centre_lon':mpi_hist_lons_SH[:,4], 'date':find_dates(mpi_hist_sids_SH,'proleptic_gregorian')})

# CMIP6 Noresm
noresm_hist_rain = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_rain_NH.npy')
noresm_hist_lats = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_lats_NH.npy')
noresm_hist_lons = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_lons_NH.npy')
noresm_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_sid_NH.csv')
meta_noresm_hist = pd.DataFrame({'sid':noresm_hist_sids.sid ,'centre_lat':noresm_hist_lats[:,4], 'centre_lon':noresm_hist_lons[:,4], 'date':find_dates(noresm_hist_sids,'noleap')})

noresm_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_rain_SH.npy')
noresm_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_lats_SH.npy')
noresm_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_lons_SH.npy')
noresm_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/NorESM2-LM/storm_rain/historical/storm_sid_SH.csv')
meta_noresm_hist_SH = pd.DataFrame({'sid':noresm_hist_sids_SH.sid ,'centre_lat':noresm_hist_lats_SH[:,4], 'centre_lon':noresm_hist_lons_SH[:,4], 'date':find_dates(noresm_hist_sids,'noleap')})

# CMIP6 IPSL
ipsl_hist_rain = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_rain_NH.npy')
ipsl_hist_lats = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_lats_NH.npy')
ipsl_hist_lons = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_lons_NH.npy')
ipsl_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_sid_NH.csv')
meta_ipsl_hist = pd.DataFrame({'sid':ipsl_hist_sids.sid ,'centre_lat':ipsl_hist_lats[:,4], 'centre_lon':ipsl_hist_lons[:,4], 'date':find_dates(ipsl_hist_sids,'gregorian')})

ipsl_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_rain_SH.npy')
ipsl_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_lats_SH.npy')
ipsl_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_lons_SH.npy')
ipsl_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/IPSL-CM6A-LR/storm_rain/historical/storm_sid_SH.csv')
meta_ipsl_hist_SH = pd.DataFrame({'sid':ipsl_hist_sids_SH.sid ,'centre_lat':ipsl_hist_lats_SH[:,4], 'centre_lon':ipsl_hist_lons_SH[:,4], 'date':find_dates(ipsl_hist_sids_SH,'gregorian')})

# CMIP6 mri
mri_hist_rain = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_rain_NH.npy')
mri_hist_lats = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_lats_NH.npy')
mri_hist_lons = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_lons_NH.npy')
mri_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_sid_NH.csv')
meta_mri_hist = pd.DataFrame({'sid':mri_hist_sids.sid ,'centre_lat':mri_hist_lats[:,4], 'centre_lon':mri_hist_lons[:,4], 'date':find_dates(mri_hist_sids,'proleptic_gregorian')})

mri_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_rain_SH.npy')
mri_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_lats_SH.npy')
mri_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_lons_SH.npy')
mri_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/MRI-ESM2-0/storm_rain/historical/storm_sid_SH.csv')
meta_mri_hist_SH = pd.DataFrame({'sid':mri_hist_sids_SH.sid ,'centre_lat':mri_hist_lats_SH[:,4], 'centre_lon':mri_hist_lons_SH[:,4], 'date':find_dates(mri_hist_sids_SH,'proleptic_gregorian')})

# CMIP6 EC-Earth
ecearth_hist_rain = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_rain_NH.npy')
ecearth_hist_lats = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_lats_NH.npy')
ecearth_hist_lons = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_lons_NH.npy')
ecearth_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_sid_NH.csv')
meta_ecearth_hist = pd.DataFrame({'sid':ecearth_hist_sids.sid ,'centre_lat':ecearth_hist_lats[:,4], 'centre_lon':ecearth_hist_lons[:,4], 'date':find_dates(ecearth_hist_sids,'proleptic_gregorian')})

ecearth_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_rain_SH.npy')
ecearth_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_lats_SH.npy')
ecearth_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_lons_SH.npy')
ecearth_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/EC-Earth3/storm_rain/historical/storm_sid_SH.csv')
meta_ecearth_hist_SH = pd.DataFrame({'sid':ecearth_hist_sids_SH.sid ,'centre_lat':ecearth_hist_lats_SH[:,4], 'centre_lon':ecearth_hist_lons_SH[:,4], 'date':find_dates(ecearth_hist_sids,'proleptic_gregorian')})




In [61]:
print(meta_CMCC_hist_NH)

def calc_nyears(meta):
	nyears = int(meta.year[len(meta.year)-1] - meta.year[0] + 1)
	return nyears




                                           sid  centre_lat  centre_lon  \
0     TC_CMCC-CM2-VHR4_historical_1979_NH_2385        8.25      112.65   
1     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       17.95      120.25   
2     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       19.25      121.05   
3     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       21.25      122.15   
4     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       23.75      123.45   
...                                        ...         ...         ...   
5231  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       14.15      127.75   
5232  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       14.95      128.35   
5233  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       56.15     -157.15   
5234  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       59.35     -155.15   
5235  TC_CMCC-CM2-VHR4_historical_2014_NH_5321       33.75      161.45   

                     date  
0     1979-04-23 00:00:00  
1     1979-06-02 00:00:00  
2     1979-06-02 00:00:00  

In [62]:
# CMIP6 CMCC_LR
cmcclr_hist_rain = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_rain_NH.npy')
cmcclr_hist_lats = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_lats_NH.npy')
cmcclr_hist_lons = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_lons_NH.npy')
cmcclr_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_sid_NH.csv')
meta_cmcclr_hist = pd.DataFrame({'sid':cmcclr_hist_sids.sid ,'centre_lat':cmcclr_hist_lats[:,4], 'centre_lon':cmcclr_hist_lons[:,4], 'date':find_dates(cmcclr_hist_sids,'365_day')})

cmcclr_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_rain_SH.npy')
cmcclr_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_lats_SH.npy')
cmcclr_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_lons_SH.npy')
cmcclr_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/CMCC-ESM2/storm_rain/historical/storm_sid_SH.csv')
meta_cmcclr_hist_SH = pd.DataFrame({'sid':cmcclr_hist_sids_SH.sid ,'centre_lat':cmcclr_hist_lats_SH[:,4], 'centre_lon':cmcclr_hist_lons_SH[:,4], 'date':find_dates(cmcclr_hist_sids_SH,'365_day')})

# CMIP6 NESM3
nesm_hist_rain = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_rain_NH.npy')
nesm_hist_lats = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_lats_NH.npy')
nesm_hist_lons = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_lons_NH.npy')
nesm_hist_sids = pd.read_csv('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_sid_NH.csv')
meta_nesm_hist = pd.DataFrame({'sid':nesm_hist_sids.sid ,'centre_lat':nesm_hist_lats[:,4], 'centre_lon':nesm_hist_lons[:,4], 'date':find_dates(nesm_hist_sids,'standard')})

nesm_hist_rain_SH = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_rain_SH.npy')
nesm_hist_lats_SH = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_lats_SH.npy')
nesm_hist_lons_SH = np.load('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_lons_SH.npy')
nesm_hist_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/NESM3/storm_rain/historical/storm_sid_SH.csv')
meta_nesm_hist_SH = pd.DataFrame({'sid':nesm_hist_sids_SH.sid ,'centre_lat':nesm_hist_lats_SH[:,4], 'centre_lon':nesm_hist_lons_SH[:,4], 'date':find_dates(nesm_hist_sids_SH,'standard')})

# load trmm
# trmm_rain = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_rain_NH.npy')
# trmm_lats = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_lats_NH.npy')
# trmm_lons = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_lons_NH.npy')
# trmm_sids = pd.read_csv('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_sid_NH.csv')
# print(imerg_sids.sid.shape)
# meta_trmm = pd.DataFrame({'sid':trmm_sids.sid ,'centre_lat':trmm_lats[:,4], 'centre_lon':trmm_lons[:,4], 'hour':trmm_sids.hour, 'year':trmm_sids.year})

# trmm_rain_SH = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_rain_SH.npy')
# trmm_lats_SH = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_lats_SH.npy')
# trmm_lons_SH = np.load('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_lons_SH.npy')
# trmm_sids_SH = pd.read_csv('/user/home/al18709/work/CMIP6/TRMM/storm_rain/storm_sid_SH.csv')
# print(trmm_sids.sid.shape)
# meta_trmm_SH = pd.DataFrame({'sid':trmm_sids.sid ,'centre_lat':trmm_lats[:,4], 'centre_lon':trmm_lons[:,4], 'hour':trmm_sids.hour})

In [92]:
# load dataset
# load data
def generate_annual_frequency(meta,rain,units_converter):
	
	# get subset of storms
	dates = meta.date
	dates_unique=dates.drop_duplicates()
	ndates = len(dates_unique)
	print(dates.shape)
	print(meta.shape)
	print(meta)
	# meta.date = pd.to_datetime(meta.date)
	tracks_grouped = meta.groupby(meta.date)
	# tracks_grouped = meta.groupby(pd.Grouper(freq='M'))
	# tracks_grouped = meta.resample(rule='M', on='date')
	print(tracks_grouped)
	# total_rain = np.zeros((ndates,100,100))
	total_rain = np.zeros((ndates))
	i=0
	drop_idx=[]
	south_idx=[]
	north_idx=[]
	# find time frequency
	print('dates unique',dates_unique.shape)
	for date in dates_unique:
		print(date)
		date_for_rain = tracks_grouped.get_group(date)
		storm_rain = rain[date_for_rain.index,:,:,0]
		print('storm_rain shape',storm_rain.shape)

		# imerg is mm/hr sampled based on ibtracs which is 3 hourly positions
		units_converter_imerg = 3

		# mswep is total rain mm/3hr sampled based on ibtracs which is 3 hourly positions interpolated from 6 hourly
		units_converter_mswep = 1

		# cmip is mm/hr sampled every 6 hourly
		units_converter_cmip = 6


		
		total_daily_rain = np.mean(storm_rain,axis=0)*units_converter
		
		

		# total_rain[i,:,:] = total_daily_rain
		print('shape of total_daily_rain sum',np.mean(total_daily_rain))
		total_rain[i] = np.mean(total_daily_rain)
		i=i+1

	
	# # get nh and sh datasets
	# composite_storm_rain_nh = composite_storm_rain[north_idx,:,:]
	# composite_storm_rain_sh = composite_storm_rain[south_idx,:,:]
	
	return total_rain,dates_unique




In [93]:
# Obs and WGAN
# total_rain,dates = generate_annual_frequency(meta,real,1)
# total_rain_x,dates_x = generate_annual_frequency(meta_og_x,np.expand_dims(real_og_x,axis=-1),1)
# ttoal_rain_wgan,dates_wgan = generate_annual_frequency(meta,pred,1)
# total_rain_og,dates_og = generate_annual_frequency(meta_og,pred_og,1)
# total_rain_og_x,dates_og_x = generate_annual_frequency(meta_og_x,pred_og_x,1)

# imerg
# set imerg=True?
# i don't think we need to multiply rain by 3 hours maybe? because its rain rate over the hour so we assume it rains at the same rate for 3 or 6 hours depending on the track?
# I think the storm track is handling the fact its rain rate and not accumulated rainfall here.
# imerg_total_rain,imerg_dates = generate_annual_frequency(meta_imerg,np.expand_dims(imerg_rain,axis=-1),3)
# nyears_imerg = calc_nyears(meta_imerg)

# trmm
# composite_trmm_NH,composite_trmm_SH,nstorms_trmm_nh,nstorms_trmm_sh = generate_annual_frequency(meta_trmm,np.expand_dims(trmm_rain,axis=-1),ibtracks=og_tracks_grouped)
# nyears_trmm = calc_nyears(meta_trmm)

# HighResMIP
total_rain_CMCC_nh,dates_CMCC_nh = generate_annual_frequency(meta_CMCC_hist_NH,np.expand_dims(CMCC_hist_NH_rain,axis=-1),6)
# nyears_CMCC_hist_NH = calc_nyears(meta_CMCC_hist_NH)
total_rain_CMCC_sh,dates_CMCC_sh = generate_annual_frequency(meta_CMCC_hist_SH,np.expand_dims(CMCC_hist_SH_rain,axis=-1),6)
# nyears_CMCC_hist_SH = calc_nyears(meta_CMCC_hist_SH)

total_rain_ecearthhr_nh,dates_exearthhr_nh = generate_annual_frequency(meta_ecearthhr_hist_NH,np.expand_dims(ecearthhr_hist_NH_rain,axis=-1),6)
# nyears_ecearthhr_hist_NH = calc_nyears(meta_ecearthhr_hist_NH)
total_rain_ecearthhr_sh,dates_exearthhr_sh = generate_annual_frequency(meta_ecearthhr_hist_SH,np.expand_dims(ecearthhr_hist_SH_rain,axis=-1),6)
# nyears_ecearthhr_hist_SH = calc_nyears(meta_ecearthhr_hist_SH)

# CMIP6
total_rain_miroc6_nh,dates_miroc6_nh = generate_annual_frequency(meta_miroc6_hist,np.expand_dims(miroc6_hist_rain,axis=-1),6)
# nyears_miroc6_hist_nh = calc_nyears(meta_miroc6_hist)
total_rain_miroc6_sh,dates_miroc6_sh = generate_annual_frequency(meta_miroc6_hist_SH,np.expand_dims(miroc6_hist_rain_SH,axis=-1),6)
# nyears_miroc6_hist_sh = calc_nyears(meta_miroc6_hist_SH)

total_rain_mpi_nh,dates_mpi_nh = generate_annual_frequency(meta_mpi_hist,np.expand_dims(mpi_hist_rain,axis=-1),6)
# nyears_mpi_hist_nh = calc_nyears(meta_mpi_hist)
total_rain_mpi_sh,dates_mpi_sh = generate_annual_frequency(meta_mpi_hist_SH,np.expand_dims(mpi_hist_rain_SH,axis=-1),6)
# nyears_mpi_hist_sh = calc_nyears(meta_mpi_hist_SH)

# total_rain_noresm_nh,dates_noresm_nh = generate_annual_frequency(meta_noresm_hist,np.expand_dims(noresm_hist_rain,axis=-1),6)
# nyears_noresm_hist_nh = calc_nyears(meta_noresm_hist)
# total_rain_noresm_nh,dates_noresm_nh = generate_annual_frequency(meta_noresm_hist_SH,np.expand_dims(noresm_hist_rain_SH,axis=-1),6)
# nyears_noresm_hist_sh = calc_nyears(meta_noresm_hist_SH)

total_rain_nesm_nh,dates_nesm_nh = generate_annual_frequency(meta_nesm_hist,np.expand_dims(nesm_hist_rain,axis=-1),6)
# nyears_nesm_hist_nh = calc_nyears(meta_nesm_hist)
total_rain_nesm_sh,dates_nesm_sh = generate_annual_frequency(meta_nesm_hist_SH,np.expand_dims(nesm_hist_rain_SH,axis=-1),6)
# nyears_nesm_hist_sh = calc_nyears(meta_nesm_hist_SH)

total_rain_ipsl_nh,dates_ipsl_nh = generate_annual_frequency(meta_ipsl_hist,np.expand_dims(ipsl_hist_rain,axis=-1),6)
# nyears_ipsl_hist_nh = calc_nyears(meta_ipsl_hist)
total_rain_ipsl_sh,dates_ipsl_sh = generate_annual_frequency(meta_ipsl_hist_SH,np.expand_dims(ipsl_hist_rain_SH,axis=-1),6)
# nyears_ipsl_hist_sh = calc_nyears(meta_ipsl_hist_SH)

total_rain_mri_nh,dates_mri_nh = generate_annual_frequency(meta_mri_hist,np.expand_dims(mri_hist_rain,axis=-1),6)
# nyears_mri_hist_nh = calc_nyears(meta_mri_hist)
total_rain_mri_sh,dates_mri_sh  = generate_annual_frequency(meta_mri_hist_SH,np.expand_dims(mri_hist_rain_SH,axis=-1),6)
# nyears_mri_hist_sh = calc_nyears(meta_mri_hist_SH)

total_rain_ecearth_nh,dates_ecearth_nh = generate_annual_frequency(meta_ecearth_hist,np.expand_dims(ecearth_hist_rain,axis=-1),6)
# nyears_ecearth_hist_nh = calc_nyears(meta_ecearth_hist)
# total_rain_ecearth_sh,dates_ecearth_sh = generate_annual_frequency(meta_ecearth_hist_SH,np.expand_dims(ecearth_hist_rain_SH,axis=-1),6)
# nyears_ecearth_hist_sh = calc_nyears(meta_ecearth_hist_SH)

total_rain_cmcclr_nh,dates_cmcclr_nh = generate_annual_frequency(meta_cmcclr_hist,np.expand_dims(cmcclr_hist_rain,axis=-1),6)
# nyears_cmcclr_hist_nh = calc_nyears(meta_cmcclr_hist)
total_rain_cmcclr_sh,dates_cmcclr_sh = generate_annual_frequency(meta_cmcclr_hist_SH,np.expand_dims(cmcclr_hist_rain_SH,axis=-1),6)
# nyears_cmcclr_hist_sh = calc_nyears(meta_cmcclr_hist_SH)

(5236,)
(5236, 4)
                                           sid  centre_lat  centre_lon  \
0     TC_CMCC-CM2-VHR4_historical_1979_NH_2385        8.25      112.65   
1     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       17.95      120.25   
2     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       19.25      121.05   
3     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       21.25      122.15   
4     TC_CMCC-CM2-VHR4_historical_1979_NH_2388       23.75      123.45   
...                                        ...         ...         ...   
5231  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       14.15      127.75   
5232  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       14.95      128.35   
5233  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       56.15     -157.15   
5234  TC_CMCC-CM2-VHR4_historical_2014_NH_5312       59.35     -155.15   
5235  TC_CMCC-CM2-VHR4_historical_2014_NH_5321       33.75      161.45   

                     date  
0     1979-04-23 00:00:00  
1     1979-06-02 00:00:00  
2     197

storm_rain shape (4, 100, 100)
shape of total_daily_rain sum 24.781770122462415
1985-09-13 00:00:00
storm_rain shape (4, 100, 100)
shape of total_daily_rain sum 25.664644562199896
1985-09-17 00:00:00
storm_rain shape (1, 100, 100)
shape of total_daily_rain sum 37.821616588104234
1985-09-18 00:00:00
storm_rain shape (2, 100, 100)
shape of total_daily_rain sum 30.31826370065345
1985-09-24 00:00:00
storm_rain shape (3, 100, 100)
shape of total_daily_rain sum 23.6400620333336
1985-09-25 00:00:00
storm_rain shape (1, 100, 100)
shape of total_daily_rain sum 22.87633543681588
1985-09-26 00:00:00
storm_rain shape (1, 100, 100)
shape of total_daily_rain sum 29.542058713534033
1985-08-28 00:00:00
storm_rain shape (1, 100, 100)
shape of total_daily_rain sum 16.285453193577563
1985-10-05 00:00:00
storm_rain shape (3, 100, 100)
shape of total_daily_rain sum 32.94403252483287
1985-10-01 00:00:00
storm_rain shape (1, 100, 100)
shape of total_daily_rain sum 39.88007721917457
1985-10-15 00:00:00
storm_

In [94]:
print(total_rain_cmcclr_sh.shape)
print(dates_cmcclr_sh.shape)
# print(total_rain_ecearth_nh.shape,dates_ecearth_nh.shape)

(565,)
(565,)


In [ ]:
# calculate number of storms for sh / nh weight ratio for final plot
# nstorms_cmip6_hist_nh = nstorms_miroc6_hist_nh + nstorms_mpi_hist_nh + nstorms_noresm_hist_nh + nstorms_nesm_hist_nh + nstorms_ipsl_hist_nh + nstorms_mri_hist_nh + nstorms_ecearth_hist_nh + nstorms_cmcclr_hist_nh
# nstorms_cmip6_hist_sh = nstorms_miroc6_hist_sh + nstorms_mpi_hist_sh + nstorms_noresm_hist_sh + nstorms_nesm_hist_sh + nstorms_ipsl_hist_sh + nstorms_mri_hist_sh + nstorms_ecearth_hist_sh + nstorms_cmcclr_hist_sh

# nstorms_hrmip_hist_nh = nstorms_CMCC_hist_nh + nstorms_ecearthhr_hist_nh
# nstorms_hrmip_hist_sh = nstorms_CMCC_hist_sh + nstorms_ecearthhr_hist_sh

# print(nstorms_cmip6_hist_nh)
# print(nstorms_cmip6_hist_sh)
# print(nstorms_cmip6_hist_sh + nstorms_cmip6_hist_nh)
# print(nstorms_cmip6_hist_nh / (nstorms_cmip6_hist_sh + nstorms_cmip6_hist_nh))
# print(nstorms_cmip6_hist_sh / (nstorms_cmip6_hist_sh + nstorms_cmip6_hist_nh))

# nstorms_cmip6_ssp585_nh = nstorms_miroc6_ssp585_nh + nstorms_mpi_ssp585_nh + nstorms_noresm_ssp585_nh + nstorms_nesm_ssp585_nh + nstorms_ipsl_ssp585_nh + nstorms_mri_ssp585_nh + nstorms_ecearth_ssp585_nh + nstorms_cmcclr_ssp585_nh
# nstorms_cmip6_ssp585_sh = nstorms_miroc6_ssp585_sh + nstorms_mpi_ssp585_sh + nstorms_noresm_ssp585_sh + nstorms_nesm_ssp585_sh + nstorms_ipsl_ssp585_sh + nstorms_mri_ssp585_sh + nstorms_ecearth_ssp585_sh + nstorms_cmcclr_ssp585_sh

In [ ]:
# define cmap
precip_cmap,precip_norm = make_cmap()

In [ ]:

fig,axes = plt.subplots(1,1,figsize=(10, 10))

# MSWEP Obs
# im_1 = axes[0,0].contourf(total_rain_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[0,0].annotate('Max: %.1f' % np.max(total_rain_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[0,0].annotate('Storms: %.0f' % nstorms_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# # axes[0,0].annotate('Storms per year: %.0f' % (int(nstorms_nh) / nyea), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# im_2 = axes[0,1].contourf(total_rain_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[0,1].annotate('Max: %.1f' % np.max(total_rain_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[0,1].annotate('Storms: %.0f' % nstorms_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# im_3 = axes[0,2].contourf(total_rain_nh_x,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[0,2].annotate('Max: %.1f' % np.max(total_rain_nh_x), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[0,2].annotate('Storms: %.0f' % nstorms_x_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# im_4 = axes[0,3].contourf(total_rain_sh_x,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[0,3].annotate('Max: %.1f' % np.max(total_rain_sh_x), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[0,3].annotate('Storms: %.0f' % nstorms_x_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

# # # IMERG Obs
# im_5 = axes[1,0].contourf(total_rain_imerg_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,0].annotate('Max: %.1f' % np.max(total_rain_imerg_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_6 = axes[1,1].contourf(total_rain_imerg_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,1].annotate('Max: %.1f' % np.max(total_rain_imerg_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# im_5b = axes[1,2].contourf(imerg_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,2].annotate('Max: %.1f' % np.max(imerg_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_6b = axes[1,3].contourf(imerg_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,3].annotate('Max: %.1f' % np.max(imerg_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# # TRMM Obs
# im_5 = axes[1,2].contourf(total_rain_trmm_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,2].annotate('Max: %.1f' % np.max(total_rain_trmm_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_6 = axes[1,3].contourf(total_rain_trmm_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,3].annotate('Max: %.1f' % np.max(total_rain_trmm_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# im_5b = axes[1,2].contourf(imerg_NH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,2].annotate('Max: %.1f' % np.max(imerg_NH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_6b = axes[1,3].contourf(imerg_SH,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[1,3].annotate('Max: %.1f' % np.max(imerg_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# HighresMIP
im = axes[0].contourf(total_rain_CMCC_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[3,0].annotate('Max: %.1f' % np.max(total_rain_CMCC_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[3,0].annotate('Storms: %.0f' % nstorms_CMCC_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# axes[3,0].annotate('Storms per year: %.0f' % (int(nstorms_CMCC_nh) / nyears_CMCC_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_6 = axes[0].contourf(total_rain_CMCC_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[3,1].annotate('Max: %.1f' % np.max(total_rain_CMCC_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[3,1].annotate('Storms: %.0f' % nstorms_CMCC_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# axes[3,1].annotate('Storms per year: %.0f' % (int(nstorms_CMCC_sh) / nyears_CMCC_sh) , xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

im_5b = axes[0].contourf(total_rain_ecearthhr_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[3,2].annotate('Max: %.1f' % np.max(total_rain_ecearthhr_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[3,2].annotate('Storms: %.0f' % nstorms_ecearthhr_nh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# axes[3,2].annotate('Storms per year: %.0f' % (int(nstorms_ecearthhr_nh) / nyears_ecearthhr_nh), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
im_6b = axes[0].contourf(total_rain_ecearthhr_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[3,3].annotate('Max: %.1f' % np.max(total_rain_ecearthhr_SH), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# axes[3,3].annotate('Storms: %.0f' % nstorms_ecearthhr_sh, xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')
# axes[3,3].annotate('Storms: %.0f' % (int(nstorms_ecearthhr_sh) / nyears_ecearthhr_SH), xy = (90,90), xytext=(3,92),fontsize=14, color='#586985')

# CMIP6 hist
im_7 = axes[0].contourf(total_rain_miroc6_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
im_8 = axes[0].contourf(total_rain_miroc6_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')

im_7b = axes[0].contourf(total_rain_mpi_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
im_8b = axes[0].contourf(total_rain_mpi_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')

# im_7c = axes[5,0].contourf(total_rain_noresm_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# im_8c = axes[5,1].contourf(total_rain_noresm_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')

im_7d = axes[0].contourf(total_rain_nesm_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
im_8c = axes[0].contourf(total_rain_nesm_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')

axes[0].contourf(total_rain_ipsl_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[0].contourf(total_rain_ipsl_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')

axes[0].contourf(total_rain_mri_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[0].contourf(total_rain_mri_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')

axes[0].contourf(total_rain_ecearth_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[7,1].contourf(total_rain_ecearth_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')

axes[0].contourf(total_rain_cmcclr_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
axes[0].contourf(total_rain_cmcclr_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')


# OG WGAN
# im_og_nh = axes[8,0].contourf(total_rain_og_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[8,0].annotate('Max: %.1f' % np.max(total_rain_og_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_og_sh = axes[8,1].contourf(total_rain_og_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[8,1].annotate('Max: %.1f' % np.max(total_rain_og_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_og_x_nh = axes[8,2].contourf(total_rain_og_x_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[8,2].annotate('Max: %.1f' % np.max(total_rain_og_x_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_og_x_sh = axes[8,3].contourf(total_rain_og_x_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[8,3].annotate('Max: %.1f' % np.max(total_rain_og_x_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# Scalar WGAN
# im_pred_nh = axes[9,0].contourf(total_rain_wgan_nh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[9,0].annotate('Max: %.1f' % np.max(total_rain_wgan_nh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')
# im_pred_sh = axes[9,1].contourf(total_rain_wgan_sh,cmap=precip_cmap,levels=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140],extend='max')
# axes[9,1].annotate('Max: %.1f' % np.max(total_rain_wgan_sh), xy = (90,90), xytext=(70,3),fontsize=14, color='#586985')

# Set the tick positions
t_p = [0,10,20,30,40,50,60,70,80,90,100]
t_l = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
for i in range(10):
	for j in range(4):
		axes[i,j].set_xticks(t_p)
		axes[i,j].set_yticks(t_p)
		# Set the tick labels
		axes[i,j].set_xticklabels(t_l)
		axes[i,j].set_yticklabels(t_l)


axes[0,0].set_title('MSWEP (nh)')
axes[0,0].set_xlabel('Longitude (degrees)')
axes[0,0].set_ylabel('Latitude (degrees)')
axes[0,1].set_title('MSWEP (SH)')
# axes[0,1].set_xlabel('Longitude (degrees)')
# add **font to change font

axes[0,2].set_title('MSWEP extreme 100 (nh)')
axes[0,3].set_title('MSWEP extreme 100 (SH)')

axes[1,0].set_title('IMERG (nh)')
axes[1,1].set_title('IMERG (SH)')
axes[1,2].set_title('IMERG extreme 100 (nh)')
axes[1,3].set_title('IMERG extreme 100 (SH)')

axes[2,0].set_title('TRMM (nh)')
axes[2,1].set_title('TRMM (SH)')
axes[2,2].set_title('TRMM extreme 100 (nh)')
axes[2,3].set_title('TRMM extreme 100 (SH)')

axes[3,0].set_title('HighResMIP CMCC Hist (nh)')
axes[3,1].set_title('HighResMIP CMCC Hist (SH)')
axes[3,2].set_title('HighResMIP EC-Earth-3P-HR Hist (nh)')
axes[3,3].set_title('HighResMIP EC-Earth-3P-HR Hist (SH)')

axes[4,0].set_title('CMIP6 MIROC6 Hist (nh)')
axes[4,1].set_title('CMIP6 MIROC6 Hist (SH)')
axes[4,2].set_title('CMIP6 MPI-ESM1-2-LR (nh)')
axes[4,3].set_title('CMIP6 MPI-ESM1-2-LR (SH)')

axes[5,0].set_title('CMIP6 NORESM hist (nh)')
axes[5,1].set_title('CMIP6 NORESM hist (SH)')
axes[5,2].set_title('CMIP6 NESM3 (nh)')
axes[5,3].set_title('CMIP6 NESM3 (SH)')

axes[6,0].set_title('CMIP6 IPSL-CM6A-LR (nh)')
axes[6,1].set_title('CMIP6 IPSL-CM6A-LR (SH)')
axes[6,2].set_title('CMIP6 MRI-ESM2-0 (nh)')
axes[6,3].set_title('CMIP6 MRI-ESM2-0 (SH)')

axes[7,0].set_title('CMIP6 EC-Earth3 (nh)')
axes[7,1].set_title('CMIP6 EC-Earth3 (SH)')
axes[7,2].set_title('CMIP6 CMCC-ESM2 (nh)')
axes[7,3].set_title('CMIP6 CMCC-ESM2 (SH)')

axes[8,0].set_title('2D WGAN (nh)')
axes[8,1].set_title('2D WGAN (SH)')
axes[8,2].set_title('2D WGAN extreme (nh)')
axes[8,3].set_title('2D WGAN extreme (SH)')

axes[9,0].set_title('Scalar WGAN applied to ERA5 (nh)')
axes[9,1].set_title('Scalar WGAN applied to ERA5 (SH)')
axes[9,2].set_title('Scalar WGAN extreme applied to ERA5 (nh)')
axes[9,3].set_title('Scalar WGAN extreme applied to ERA5 (SH)')




plt.colorbar(im_2,ticks=[0,20,40,60,80,100,120,140],cax=axes[0,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[1,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[2,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[3,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[4,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[5,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[6,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[7,4])
plt.colorbar(im_5,ticks=[0,20,40,60,80,100,120,140],cax=axes[8,4])


plt.savefig('total_rain_rain5.png',bbox_inches='tight')

In [ ]:
# save everything
fig_dir = '/user/home/al18709/work/figures/figure_1/'



# HighresMIP
np.save(fig_dir + 'composite_CMCC_hist_NH.npy',composite_CMCC_hist_NH)
np.save(fig_dir + 'composite_CMCC_hist_SH.npy', composite_CMCC_hist_SH)

np.save(fig_dir + 'composite_ecearthhr_hist_NH.npy',composite_ecearthhr_hist_NH)
np.save(fig_dir + 'composite_ecearthhr_hist_SH.npy',composite_ecearthhr_hist_SH)

# CMIP6 hist
np.save(fig_dir + 'composite_miroc6_hist_nh.npy',composite_miroc6_hist_nh)
np.save(fig_dir + 'composite_miroc6_hist_sh.npy',composite_miroc6_hist_sh)

np.save(fig_dir + 'composite_mpi_hist_nh.npy',composite_mpi_hist_nh)
np.save(fig_dir + 'composite_mpi_hist_sh.npy',composite_mpi_hist_sh)

np.save(fig_dir + 'composite_noresm_hist_nh.npy',composite_noresm_hist_nh)
np.save(fig_dir + 'composite_noresm_hist_sh.npy',composite_noresm_hist_sh)

# np.save(fig_dir + composite_taiesm_hist_nh,'composite_taiesm_hist_nh.npy')
# np.save(fig_dir + composite_taiesm_hist_sh,'composite_taiesm_hist_sh.npy')

np.save(fig_dir + 'composite_ipsl_hist_nh.npy',composite_ipsl_hist_nh)
np.save(fig_dir + 'composite_ipsl_hist_sh.npy',composite_ipsl_hist_sh)

np.save(fig_dir + 'composite_mri_hist_nh.npy',composite_mri_hist_nh)
np.save(fig_dir + 'composite_mri_hist_sh.npy',composite_mri_hist_sh)

np.save(fig_dir + 'composite_ecearth_hist_nh.npy',composite_ecearth_hist_nh)
np.save(fig_dir + 'composite_ecearth_hist_sh.npy',composite_ecearth_hist_sh)

np.save(fig_dir + 'composite_cmcclr_hist_nh.npy',composite_cmcclr_hist_nh)
np.save(fig_dir + 'composite_cmcclr_hist_sh.npy',composite_cmcclr_hist_sh)

# np.save(fig_dir + composite_bcc_hist_nh,'composite_bcc_hist_nh.npy')
# np.save(fig_dir + composite_bcc_hist_sh,'composite_bcc_hist_sh.npy')

np.save(fig_dir + 'composite_nems_hist_nh.npy',composite_nesm_hist_nh)
np.save(fig_dir + 'composite_nems_hist_sh.npy',composite_nesm_hist_sh)


# OG WGAN
# np.save(composite_og_nh,'composite_og_nh.npy')
# np.save(composite_og_sh,'composite_og_sh.npy')
# np.save(composite_og_x_nh,'composite_og_x_nh.npy')
# np.save(composite_og_x_nh,'composite_og_x_nh.npy')
# np.save(composite_og_x_sh,'composite_og_x_sh.npy')

# # Scalar WGAN
# np.save(composite_wgan_nh,'composite_wgan_nh.npy')
# np.save(composite_wgan_sh,'composite_wgan_sh.npy')
